# *TP1 - DEEP LEARNING*

### 2.5 Application à MNIST

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import numpy as np
import matplotlib.pyplot as plt

#### Écriver votre propre code, pour charger la dataset MNIST depuis pytorch et utiliser l'architecture de notre réseau de neurones précédement dévelopé  pour classifier les différents chifffre de 0 à 10. 

In [ ]:
# Chargement MNIST, définition modèle, entraînement et évaluation
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# --- paramètres ---
batch_size = 128
epochs = 5
lr = 1e-3
model_path = 'mnist_cnn.pth'
# --- dataset ---
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_ds = torchvision.datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_ds  = torchvision.datasets.MNIST(root='data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
print(f'Train size: {len(train_ds)}, Test size: {len(test_ds)}, Device: {device}')
# --- modèle simple CNN ---
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(64*14*14, 128)
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
model = SimpleCNN().to(device)
# --- opt / loss ---
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
# --- fonctions utilitaires ---
def train_one_epoch(loader, model, opt, criterion, device):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        opt.step()
        total_loss += float(loss.item()) * xb.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total += xb.size(0)
    return total_loss / total, correct / total
def evaluate(loader, model, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            loss = criterion(logits, yb)
            total_loss += float(loss.item()) * xb.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += xb.size(0)
    return total_loss / total, correct / total
# --- entraînement ---
train_losses, train_accs = [], []
test_losses, test_accs = [], []
for epoch in range(1, epochs+1):
    tr_loss, tr_acc = train_one_epoch(train_loader, model, optimizer, criterion, device)
    te_loss, te_acc = evaluate(test_loader, model, criterion, device)
    train_losses.append(tr_loss); train_accs.append(tr_acc)
    test_losses.append(te_loss); test_accs.append(te_acc)
    print(f'Epoch {epoch}/{epochs} — train loss: {tr_loss:.4f}, train acc: {tr_acc:.4f} — test loss: {te_loss:.4f}, test acc: {te_acc:.4f}')
# --- sauvegarde modèle ---
torch.save(model.state_dict(), model_path)
print('Saved model to', model_path)
# --- affichage de quelques prédictions ---
import matplotlib.pyplot as plt
model.eval()
xb, yb = next(iter(test_loader))
xb, yb = xb.to(device), yb.to(device)
with torch.no_grad():
    logits = model(xb[:16])
    preds = logits.argmax(dim=1).cpu().numpy()
images = xb[:16].cpu().numpy()
labels = yb[:16].cpu().numpy()
fig, axes = plt.subplots(4,4, figsize=(6,6))
for i, ax in enumerate(axes.flat):
    ax.imshow(images[i].squeeze(), cmap='gray')
    ax.set_title(f'p:{preds[i]} / t:{labels[i]}')
    ax.axis('off')
plt.tight_layout()
plt.show()